# NO2 Data Collection and CSV Export
Mengambil data NO2 dari Sentinel-5P dan menyimpannya dalam format CSV.

In [24]:
# Install required packages
import sys
import subprocess

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package, "--user"])

# Install basic required packages
required_packages = ["xarray", "pandas", "openeo"]

for package in required_packages:
    try:
        print(f"Installing {package}...")
        install_package(package)
    except Exception as e:
        print(f"Error installing {package}: {e}")

# Import required libraries
import openeo
import os
import xarray as xr
import pandas as pd

Installing xarray...
Installing pandas...
Installing pandas...
Installing openeo...
Installing openeo...


In [25]:
# Connect to openEO backend
connection = openeo.connect("openeo.dataspace.copernicus.eu").authenticate_oidc()

Authenticated using refresh token.


In [26]:
# Define area of interest using polygon coordinates
coordinates = [
    [112.45390300848345, -6.880960455552255],
    [112.43322017927579, -6.957502115216954],
    [112.36628008244071, -6.986970395002288],
    [112.47579357551535, -7.000963701010065],
    [112.51119020298182, -7.015313325230437],
    [112.50321358450208, -7.030157252634282],
    [112.5211610390948, -7.045495833494314],
    [112.52315569364674, -7.035596963077623],
    [112.54160082195767, -7.050436746853933],
    [112.35855150090839, -7.303858335073286],
    [112.44372211514786, -7.308673680530134],
    [112.47199724089893, -7.39682101338893],
    [112.5770747403231, -7.358772723274527],
    [112.5757555441208, -7.234521445420839],
    [112.66196464264533, -7.199776078051002],
    [112.67345149706591, -7.177045125146265],
    [112.63161608651626, -7.117904371015911],
    [112.65093217073729, -7.049742216135584],
    [112.64270640054463, -7.010328782330575],
    [112.60116029304749, -6.952451741742493],
    [112.59267970095755, -6.916539431326001],
    [112.61215079634741, -6.9160438055054385],
    [112.60516117236227, -6.872922372439419],
    [112.57370780557892, -6.877383653446103],
    [112.57420697159387, -6.844173503566822],
    [112.52378198399708, -6.862513179003713],
    [112.54175530827729, -6.879366113425078],
    [112.5098058937109, -6.905639092692496],
    [112.45338989252252, -6.880855590245233]
]

# Find bounding box from coordinates
lons = [coord[0] for coord in coordinates]
lats = [coord[1] for coord in coordinates]

bbox = {
    "west": min(lons),
    "south": min(lats),
    "east": max(lons),
    "north": max(lats)
}

# Print the area coverage
print("Area coverage:")
print(f"Longitude: {bbox['west']}°E to {bbox['east']}°E")
print(f"Latitude: {bbox['south']}°N to {bbox['north']}°N")

Area coverage:
Longitude: 112.35855150090839°E to 112.67345149706591°E
Latitude: -7.39682101338893°N to -6.844173503566822°N


In [29]:
# Create a datacube with daily data
print("Loading data collection...")
cube = connection.load_collection(
    "SENTINEL_5P_L2",
    temporal_extent=["2021-09-20", "2025-09-20"],
    spatial_extent=bbox,
    bands=["NO2"]
)

# Add basic filtering to reduce data volume
cube = cube.filter_bands(["NO2"])

# Perform daily aggregation
print("Aggregating data to daily values...")
cube = cube.aggregate_temporal_period(
    period="day",  # Agregasi per hari
    reducer="mean"  # Menggunakan nilai rata-rata untuk setiap hari
)

print("Processing complete. Ready for download.")

Loading data collection...
Aggregating data to daily values...
Processing complete. Ready for download.
Aggregating data to daily values...
Processing complete. Ready for download.


In [30]:
# Create results directory if it doesn't exist
os.makedirs("results", exist_ok=True)

# Download the data
try:
    print("Downloading data...")
    cube.download("results/no2_gresik.nc")
    print("Data successfully downloaded to results/no2_gresik.nc")
    
except Exception as e:
    print(f"Error downloading data: {e}")
    
    # Try batch processing as alternative
    try:
        print("\nTrying batch processing...")
        batch_job = cube.execute_batch(outputfile="results/no2_gresik.nc")
        
        while True:
            status = batch_job.status()
            print(f"Job status: {status}")
            if status == 'finished':
                print("Download completed successfully")
                break
            elif status == 'error':
                raise Exception("Batch job failed")
    
    except Exception as e2:
        print(f"All download attempts failed.")
        print(f"Error: {e2}")
        raise

Data successfully downloaded to results/no2_gresik.nc
Data successfully downloaded to results/no2_gresik.nc
